In [97]:
# Import dependencies
import datetime
import dateutil
from openclsim.appendix import get_event_log
import openclsim.model as model
import openclsim.plugins as plugins
import simpy

In [98]:
'''
This cell is only for illustration purposes. In this case we 
start of by generating sample data. Usually, this is provided
for a site specific location.
'''

# Import extra dependencies 
import pandas as pd
import numpy as np  

# Generate sample dates
dates = np.arange(
    start=datetime.datetime(2021, 1, 1),
    stop=datetime.datetime(2021, 1, 10),
    step=datetime.timedelta(hours=1)
)

# Generate sample data
hs = pd.DataFrame(
    {'date': dates, 'hs': np.random.random(size=len(dates))}
).set_index('date')

tp = pd.DataFrame(
    {'date': dates, 'tp': np.random.random(size=len(dates))}
).set_index('date')

# Export to .csv file
hs.to_csv("./sample_data/hs.csv")
tp.to_csv("./sample_data/tp.csv")

In [99]:
# Create a simulation environment
class SimulationEnvironment(object):
    """
    A OpenCLSim simulation object.

    Use the class methods to define the activities and start the
    simulation. In this example, a basic activity is modelled using 
    the `model.BasicActivity` class and the length of the activity is
    extended using a random delay with the `model.HasDelayPlugin`. For
    further reference, check the `DelayBasicActivity` class below.

    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.
    """
    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)
        self.registry = {}

        # Create offshore env
        self.oe = plugins.OffshoreEnvironment()
        self.oe.store_information(var='hs', filename='./sample_data/hs.csv')
        self.oe.store_information(var='tp', filename='./sample_data/tp.csv')

    def define_activities(self):
        """Use this function to define your activities."""

        # Define activity
        activity = BasicActivity(
            env=self.env, 
            registry=self.registry,
            name="A basic activity",
            limit_expr=lambda hs, tp: (hs > 0.5) | (tp > 0.5),
            offshore_environment=self.oe,
            duration=3600 * 3
        )

        # Register the activities
        model.register_processes([activity]) 
        
        # Return the base activity for logging purposes
        return [activity]  
        
    def execute_simulation(self):
        """Function starts the simulation."""
        self.activities = self.define_activities()
        self.env.run()
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """Function returns the event log."""
        return get_event_log(activity_list=self.activities)

    @property
    def project_length(self):
        """Function returns the project length."""
        log = self.event_log
        dt = (log.Timestamp.iloc[-1] - log.Timestamp.iloc[0])
        return dt.total_seconds() / (3600)


class BasicActivity(plugins.HasRequestWindowPluginActivity, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [100]:
sim = SimulationEnvironment(start_date=datetime.datetime(2021, 1, 1))
res = sim.execute_simulation()
sim.event_log

,Timestamp,ActivityState,Description,ActivityID
0,2021-01-01 00:00:00,WAIT_START,A basic activity,c79cc044-029d-40c1-a204-1def918d247c
1,2021-01-02 11:00:00,WAIT_STOP,A basic activity,c79cc044-029d-40c1-a204-1def918d247c
2,2021-01-02 11:00:00,START,A basic activity,c79cc044-029d-40c1-a204-1def918d247c
3,2021-01-02 14:00:00,STOP,A basic activity,c79cc044-029d-40c1-a204-1def918d247c
